In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from utils import *
from usad import *
from data_loader import *
from thresholding import *
from eval import *
from tools import *
import torch.utils.data as data_utils

In [2]:
!nvidia-smi -L
device = get_default_device()

GPU 0: NVIDIA A100-SXM4-80GB (UUID: GPU-273b9a8a-5f5c-35c5-7e84-56e43027fa11)


In [4]:
PSM = PSMSegLoader('../TAD-Analysis/dataset/PSM', 100)
train_data, vali_data, test_data, test_labels = obtain_data(PSM)
test_labels = PSM.test_labels.reshape(-1)
del PSM

In [3]:
SMD = SMDSegLoader('../TAD-Analysis/dataset/SMD', 100)
train_data, vali_data, test_data, test_labels = obtain_data(SMD)
del SMD

In [3]:
MSL = MSLSegLoader('../TAD-Analysis/dataset/MSL', 100)
train_data, vali_data, test_data, test_labels = obtain_data(MSL)
del MSL

In [ ]:
SWAT = SWATSegLoader('../TAD-Analysis/dataset/SWaT', 100)
train_data, vali_data, test_data, test_labels = obtain_data(SWAT)
test_labels = test_labels.reshape(-1)
del SWAT

In [3]:
WADI = WADISegLoader('../TAD-Analysis/dataset/WADI', 100)
train_data, vali_data, test_data, test_labels = obtain_data(WADI)
test_labels = test_labels.reshape(-1)
del WADI

In [3]:
SMAP = SMAPSegLoader('../TAD-Analysis/dataset/SMAP', 100)
train_data, vali_data, test_data, test_labels = obtain_data(SMAP)
del SMAP

In [3]:
SWAN = NIPS_TS_SwanSegLoader('../TAD-Analysis/dataset/NIPS_TS_Swan', 100)
train_data, vali_data, test_data, test_labels = obtain_data(SWAN)
del SWAN

train: (60000, 38)
test: (60000, 38)


In [3]:
GECCO = NIPS_TS_SwanSegLoader('../TAD-Analysis/dataset/NIPS_TS_Water', 100)
train_data, vali_data, test_data, test_labels = obtain_data(GECCO)
del GECCO

train: (69260, 9)
test: (69261, 9)


### Windows

In [4]:
normal = pd.DataFrame(train_data)
attack = pd.DataFrame(test_data)

In [5]:
window_size=10

In [6]:
windows_normal=normal.values[np.arange(window_size)[None, :] + np.arange(normal.shape[0]-window_size)[:, None]]
windows_normal.shape

(69250, 10, 9)

In [7]:
windows_attack=attack.values[np.arange(window_size)[None, :] + np.arange(attack.shape[0]-window_size)[:, None]]
windows_attack.shape

(69251, 10, 9)

## Training

In [8]:
BATCH_SIZE =  7919
N_EPOCHS = 100
hidden_size = 100

w_size=windows_normal.shape[1]*windows_normal.shape[2]
z_size=windows_normal.shape[1]*hidden_size

windows_normal_train = windows_normal[:int(np.floor(.8 *  windows_normal.shape[0]))]
windows_normal_val = windows_normal[int(np.floor(.8 *  windows_normal.shape[0])):int(np.floor(windows_normal.shape[0]))]

train_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(windows_normal_train).float().view(([windows_normal_train.shape[0],w_size]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

val_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(windows_normal_val).float().view(([windows_normal_val.shape[0],w_size]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
    torch.from_numpy(windows_attack).float().view(([windows_attack.shape[0],w_size]))
) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

model = UsadModel(w_size, z_size)
model = to_device(model,device)

In [9]:
history = training(N_EPOCHS, model, train_loader, val_loader)

Epoch [0], val_loss1: 1.8936, val_loss2: 1.8909
Epoch [1], val_loss1: 1.8724, val_loss2: -0.0134
Epoch [2], val_loss1: 1.8411, val_loss2: -0.6397
Epoch [3], val_loss1: 1.8077, val_loss2: -0.9349
Epoch [4], val_loss1: 1.7842, val_loss2: -1.1015
Epoch [5], val_loss1: 1.7705, val_loss2: -1.2070
Epoch [6], val_loss1: 1.7647, val_loss2: -1.2834
Epoch [7], val_loss1: 1.7642, val_loss2: -1.3439
Epoch [8], val_loss1: 1.7663, val_loss2: -1.3934
Epoch [9], val_loss1: 1.7690, val_loss2: -1.4335
Epoch [10], val_loss1: 1.7726, val_loss2: -1.4675
Epoch [11], val_loss1: 1.7762, val_loss2: -1.4964
Epoch [12], val_loss1: 1.7797, val_loss2: -1.5214
Epoch [13], val_loss1: 1.7831, val_loss2: -1.5432
Epoch [14], val_loss1: 1.7868, val_loss2: -1.5628
Epoch [15], val_loss1: 1.7901, val_loss2: -1.5801
Epoch [16], val_loss1: 1.7944, val_loss2: -1.5967
Epoch [17], val_loss1: 1.8001, val_loss2: -1.6133
Epoch [18], val_loss1: 1.8023, val_loss2: -1.6253
Epoch [19], val_loss1: 1.8045, val_loss2: -1.6365
Epoch [20],

In [10]:
torch.save({
            'encoder': model.encoder.state_dict(),
            'decoder1': model.decoder1.state_dict(),
            'decoder2': model.decoder2.state_dict()
            }, "model.pth")

In [11]:
checkpoint = torch.load("model.pth")

model.encoder.load_state_dict(checkpoint['encoder'])
model.decoder1.load_state_dict(checkpoint['decoder1'])
model.decoder2.load_state_dict(checkpoint['decoder2'])

<All keys matched successfully>

In [12]:
results=testing(model, test_loader)

In [13]:
labels = test_labels
windows_labels=[]
for i in range(len(labels)-window_size):
    windows_labels.append(list(np.int_(labels[i:i+window_size])))

In [14]:
y_test = [1.0 if (np.sum(window) > 0) else 0 for window in windows_labels]

In [15]:
y_pred=np.concatenate([torch.stack(results[:-1]).flatten().detach().cpu().numpy(),
                              results[-1].flatten().detach().cpu().numpy()])

In [16]:
print("ROC-AUC:", calculate_roc_auc(y_test, y_pred))
print("PR-AUC:", calculate_pr_auc(y_test, y_pred))

ROC-AUC: 0.662937049900397
PR-AUC: 0.22699484521418872
